``` bash
# in terminal
conda env create -n gui --file=env.yml
conda activate gui
```

In [2]:
from pathlib import Path
import numpy as np
import soundfile as sf
from scipy.signal import spectrogram
from decord import VideoReader, cpu
import fastplotlib as fpl
from ipywidgets import VBox, HBox
import threading
import time
import cv2

In [3]:
video_path = Path("./video_center_19.mp4").resolve()
audio_path = Path("./channel_0_19.wav").resolve()

video_reader = VideoReader(str(video_path), ctx=cpu(0))
video_reader

#### any more than 500 frames fries my computer

In [4]:
movie_npy = video_reader[:500].asnumpy()
first_frame = video_reader[0].asnumpy()

In [8]:
## Load audio ##
# Load the audio file using soundfile.
audio_data, fps = sf.read(audio_path, dtype='float32')

In [9]:
global fps # for image widget, probably dont need this

In [12]:
def make_specgram(audio, fps):
    f, t, spec = spectrogram(audio, fs=fps, nfft=512, nperseg=512, noverlap=256, return_onesided=True)
    # Remove the 0-frequency bin and flip the frequency axis so that high frequencies are at the top.
    f = f[1:][::-1]
    spec = np.flip(spec[1:], axis=0)
    spec = np.log(np.abs(spec) + 1e-12).astype(np.float32)
    return t, f, spec

In [13]:
# Compute the spectrogram from audio file.
t_spec, f_spec, spec_data = make_specgram(audio_data, fps)
print(t_spec.shape)
print(" - Spectrogram computed with shape:", spec_data.shape)
print(" - First video frame shape:", first_frame.shape)
print('video frames:', len(video_reader))
print(movie_npy.shape)

(175754,)
 - Spectrogram computed with shape: (256, 175754)
 - First video frame shape: (1200, 1600, 3)
video frames: 10799
(500, 1200, 1600, 3)


In [16]:
initial_window = np.zeros((256, 488))
initial_window.shape

(256, 488)

In [21]:
initial_window = spec_data[:, :488]
initial_window.shape

(256, 488)

In [29]:
spectrogram_plot = fpl.Figure(size=(700, 300))
spectrogram_plot[0,0].toolbar = False

spectrogram_plot[0, 0].add_image(data=initial_window, name="spectrogram")
select = spectrogram_plot[0,0].graphics[0].add_linear_selector()
    
video_widget = fpl.ImageWidget([movie_npy], rgb=True, figure_kwargs={"size": (700, 360), "shape": (1, 1)})
video_widget.add_event_handler(get_spect_window, "current_index")
video_widget.figure[0,0].toolbar=False

#video_widget.show()
VBox([spectrogram_plot.show(maintain_aspect = False),video_widget.show()])

RFBOutputContext()

RFBOutputContext()

/home/flynn/miniforge3/envs/fpl_gily/lib/python3.10/site-packages/fastplotlib/graphics/_features/_base.py:18: UserWarning: casting float64 array to float32
  warn(f"casting {array.dtype} array to float32")


LinearSelector @ 0x7d33f95d5f00

In [19]:
def get_spect_window(ev):
    t_frame = ev['t'] 
    t_sec = t_frame / 30 
    
    window_duration = 1.0  
    dt = t_spec[1] - t_spec[0]  
    window_size = int(window_duration / dt) 

    center_idx = np.searchsorted(t_spec, t_sec)  
    half_window=window_size//2

    start_idx = max(0, center_idx - half_window)
    end_idx = min(len(t_spec), center_idx + half_window)

    print(f"Frame: {t_frame}, Time (sec): {t_sec}, Index: {center_idx}, Window: ({start_idx}, {end_idx})") 

    spectrogram_slice = spec_data[:, start_idx:end_idx]

    # Pad if necessary to maintain fixed size
    if spectrogram_slice.shape[1] < window_size:
        pad_width = window_size - spectrogram_slice.shape[1]
        spectrogram_slice = np.pad(
            spectrogram_slice, ((0, 0), (0, pad_width)), mode="constant"
        )

    spectrogram_plot[0, 0].graphics[0].data = spectrogram_slice

In [132]:
spectrogram_plot[0, 0].graphics[0].reset_vmin_vmax()

In [19]:
# --- Parameters ---
window_duration = 5.0  # Duration (in seconds) of the moving spectrogram window
fixed_spec_width = 320  # Fixed width (in pixels) for the displayed spectrogram window
@video_widget.add_event_handler('current_index')
def get_moving_window(t_ms, t_spec):
    """
    Given the full spectrogram (spec_data) and its time bins (t_spec),
    extract a window (slice) of duration 'window_duration' seconds centered 
    around time t_ms (in milliseconds). Returns a slice of spec_data.
    """
    t_sec = t_ms / 1000.0
    if t_sec < window_duration/2:
        start_idx = 0
        end_idx = np.searchsorted(t_spec, window_duration)
    elif t_sec > t_spec[-1] - window_duration/2:
        diff = t_spec[1] - t_spec[0]
        end_idx = len(t_spec)
        start_idx = max(0, end_idx - int(window_duration / diff))
    else:
        start_idx = np.searchsorted(t_spec, t_sec - window_duration/2)
        end_idx = np.searchsorted(t_spec, t_sec + window_duration/2)
    return spec_data[:, start_idx:end_idx]

def update_frame():
    start_time = time.time()
    while True:
        elapsed = time.time() - start_time
        if elapsed > total_duration:
            break
        frame_index = int(elapsed * fps)
        if frame_index >= movie_npy.shape[0]:
            break
        
        # Update the video widget with the current frame.
        video_widget.data = movie_npy[frame_index]
        
        # For the spectrogram, compute current time in ms.
        current_ms = elapsed * 1000
        window = get_moving_window(current_ms, spec_data, t_spec, window_duration)
        # Resize the extracted window horizontally to have a fixed width.
        window_resized = cv2.resize(window, (fixed_spec_width, window.shape[0]))
        # Update the spectrogram image data.
        img.data = window_resized
        
        # Determine time boundaries for the current window.
        if elapsed < window_duration/2:
            window_start = 0
            window_end = window_duration
        elif elapsed > t_spec[-1] - window_duration/2:
            window_start = t_spec[-1] - window_duration
            window_end = t_spec[-1]
        else:
            window_start = elapsed - window_duration/2
            window_end = elapsed + window_duration/2
        
        # Compute the relative position of the current time within the window,
        # then convert to pixel position.
        rel_pos = (elapsed - window_start) / window_duration
        x_pos = rel_pos * fixed_spec_width
        
        # Update the vertical line indicator (if you added one).
        # Assuming spec_line is the line object added to spectrogram_plot.
        #spec_line.data = ([x_pos, x_pos], [0, window_resized.shape[0]])
        
        time.sleep(1.0 / fps)

# --- Start the update loop in a separate thread so it runs in the background ---
#VBox([video_widget.show(), spectrogram_plot1.show])

threading.Thread(target=update_frame, daemon=True).start()


Exception in thread Thread-11 (update_frame):
Traceback (most recent call last):
  File "C:\Users\gg3065\AppData\Local\miniforge3\envs\fastplotlib-env\Lib\threading.py", line 1041, in _bootstrap_inner
    self.run()
    ~~~~~~~~^^
  File "C:\Users\gg3065\AppData\Local\miniforge3\envs\fastplotlib-env\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
    ~~~~~~~~~~~~~~~~~~~~~^^^^^^
  File "C:\Users\gg3065\AppData\Local\miniforge3\envs\fastplotlib-env\Lib\threading.py", line 992, in run
    self._target(*self._args, **self._kwargs)
    ~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\gg3065\AppData\Local\Temp\ipykernel_37228\1635516864.py", line 39, in update_frame
    video_widget.data = movie_npy[frame_index]
    ^^^^^^^^^^^^^^^^^
AttributeError: property 'data' of 'ImageWidget' object has no setter


In [ ]:
# Add a linear selector for interactive exploration.
#img.add_linear_selector()
#@video_widget.add_event_handler("current_index")
# def update_spectrogram(event):
#     window_width_sec = 5;
#     curr_frame = event["t"]
#     curr_sec = curr_frame/fps

#     # start and end of window
#     time_idx_start = np.argmin(np.abs(t_spec - curr_sec-window_width_sec/2))
#     time_idx_end = np.argmin(np.abs(t_spec - curr_sec+window_width_sec/2))
#     time_idx = np.argmin(np.abs(t_spec - curr_sec)) # do we need this?
    
#     test = spec_data[:, time_idx_start:time_idx_end]
#     spectrogram_plot.figure[0,0].data = test